### King County Request | December 2022

"Total onroad vehicle emissions and activity data within each jurisdiction's geographic boundary for the calendar years 2019 and 2020. If available, we would also request projections over time going to 2050. Only aggregate data is needed, broken down by the vehicle mode and fuel type, as shown in the table. If there is data that doesn't fit into the listed categories, please let us know."

Requested for all King County cities, King County total and King County unicorporated areas.

Years required include 2018, 2019, 2020, 2030, 2040, 2050. Interpolation between 2018 and 2030 are used to provide 2019 and 2020 estimates. 


A script was created to address the data request, which allows standard Soundcast network outputs to be segmented to the jurisdiction level. The "aq_tool" notebook processes Soundcast output for a list of provided cities and reports emissions and VMT by light, medium, and heavy vehicles. 

This notebook compiles and organizes the results of the aq_tool.

In [1]:
import os
import pandas as pd

In [9]:
# Location of output from aq_tool
output_dir = r'C:\Workspace\travel-modeling\aq_tool\output\HOLD\king'

In [10]:
year = '2018'
df_18 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2030'
df_30 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2040'
df_40 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

year = '2050'
df_50 = pd.read_csv(os.path.join(output_dir,year,year+'_summary.csv'), index_col='Unnamed: 0')

In [11]:
# Create interpolations
df = df_18.merge(df_30, on=['pollutant_name','city','veh_type'], suffixes=['_2018','_2030'], how='outer')

# Merge 2040
year = '2040'
df = df.merge(df_40, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)

# Merge 2050
year = '2050'
df = df.merge(df_50, on=['pollutant_name','city','veh_type'], how='outer')
df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                    'vmt': 'vmt_'+year}, inplace=True)


# df.head()

In [12]:
def interpolate(df, start_year, end_year, interpolated_year, var):
    """Interpolate values from two cols of data given start, end, and target (interpolated) years."""
    
    # Calculate annual rate of change based on difference between values at start and end year, divided number of years difference
    df['annual_'+var+'_change'] = (df[var+'_'+str(end_year)] - df[var+'_'+str(start_year)])/(end_year-start_year)
    # Calculate 
    df[var+'_'+str(interpolated_year)] = df[var+'_'+str(start_year)]+df['annual_'+var+'_change']*(interpolated_year-start_year)

    return df                                 

In [14]:
df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2021, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2022, 'vmt')
df_vmt = interpolate(df, 2018, 2030, 2023, 'vmt')


In [15]:
df_vmt = df_vmt[['city','pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020',
                 'vmt_2021','vmt_2022','vmt_2023','vmt_2030','vmt_2040','vmt_2050']]

# VMT is duplicated for each pollutant total; Select only the first set of rows
df_vmt = df_vmt.drop('pollutant_name', axis=1)
df_vmt = df_vmt.groupby(['city','veh_type']).first()

# df_vmt.to_csv(r'C:\Workspace\aq_tool\output\king_county_cities_vmt.csv')
df_vmt

vmt_2018      vmt_2019      vmt_2020      vmt_2021  \
city         veh_type                                                           
Algona       heavy     1.113933e+04  1.128446e+04  1.142959e+04  1.157472e+04   
             light     1.461125e+05  1.465700e+05  1.470276e+05  1.474852e+05   
             medium    1.097297e+04  1.090523e+04  1.083749e+04  1.076974e+04   
Auburn       heavy     6.583577e+04  6.668774e+04  6.753970e+04  6.839166e+04   
             light     1.822554e+06  1.835592e+06  1.848629e+06  1.861666e+06   
...                             ...           ...           ...           ...   
Woodinville  light     3.616180e+05  3.646699e+05  3.677218e+05  3.707737e+05   
             medium    1.362189e+04  1.377775e+04  1.393361e+04  1.408947e+04   
Yarrow Point heavy     4.266505e+01  4.320659e+01  4.374813e+01  4.428966e+01   
             light     2.246827e+04  2.265685e+04  2.284543e+04  2.303401e+04   
             medium    2.276704e+03  2.305297e+03  2.333890e+03  2.362483e+03   

                           vmt_2022      vmt_2023      vmt_2030      vmt_2040  \
city         veh_type                                                           
Algona       heavy     1.171985e+04  1.186498e+04  1.288089e+04  1.496611e+04   
             light     1.479427e+05  1.484003e+05  1.516033e+05  1.584998e+05   
             medium    1.070200e+04  1.063426e+04  1.016006e+04  9.915234e+03   
Auburn       heavy     6.924362e+04  7.009558e+04  7.605932e+04  8.691797e+04   
             light     1.874703e+06  1.887741e+06  1.979001e+06  2.090614e+06   
...                             ...           ...           ...           ...   
Woodinville  light     3.738256e+05  3.768775e+05  3.982409e+05  4.249999e+05   
             medium    1.424534e+04  1.440120e+04  1.549223e+04  1.604855e+04   
Yarrow Point heavy     4.483120e+01  4.537273e+01  4.916348e+01  5.309566e+01   
             light     2.322259e+04  2.341117e+04  2.473122e+04  2.837550e+04   
             medium    2.391076e+03  2.419669e+03  2.619821e+03  2.931864e+03   

                           vmt_2050  
city         veh_type                
Algona       heavy     1.648759e+04  
             light     1.661426e+05  
             medium    9.499568e+03  
Auburn       heavy     9.677269e+04  
             light     2.220955e+06  
...                             ...  
Woodinville  light     4.511968e+05  
             medium    1.610423e+04  
Yarrow Point heavy     6.218592e+01  
             light     3.058786e+04  
             medium    3.147581e+03  

[117 rows x 9 columns]

In [57]:
# Get county totals by vehicle type
df_city_tot_vmt = df_vmt.groupby('veh_type').sum().reset_index()

#### Compile Emissions

In [17]:
df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2021, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2022, 'total_daily_tons')
df_emissions = interpolate(df, 2018, 2030, 2023, 'total_daily_tons')

In [18]:
df_emissions = df_emissions[['city','pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
    'total_daily_tons_2020','total_daily_tons_2021','total_daily_tons_2022','total_daily_tons_2023','total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]
df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent'])]

In [60]:
df_emissions.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\king_county_cities_emissions.csv', index=False)

In [61]:
# Get county totals by vehicle type
df_city_tot_emissions = df_emissions.drop('city', axis=1).groupby(['pollutant_name','veh_type']).sum().reset_index()

### Compile Totals for County and Unincorporated Areas

Get county level results from the outputs of the regional results

In [15]:
def get_county_emissions(run_dir, county=None):
    
    # Get county level totals
    df_interzonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\interzonal_emissions.csv'))
    df_intrazonal = pd.read_csv(os.path.join(run_dir,r'outputs\emissions\intrazonal_emissions.csv'))
    start_emissions_df = pd.read_csv(os.path.join(run_dir, r'outputs\emissions\start_emissions.csv'))
    df_inter_group = df_interzonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_inter_group.rename(columns={'tons_tot': 'interzonal_tons'}, inplace=True)
    df_intra_group = df_intrazonal.groupby(['pollutantID','veh_type','county']).sum()[['tons_tot','vmt']].reset_index()
    df_intra_group.rename(columns={'tons_tot': 'intrazonal_tons'}, inplace=True)
    df_start_group = start_emissions_df.groupby(['pollutantID','veh_type','county']).sum()[['start_tons']].reset_index()

    from emissions import finalize_emissions, pollutant_map
    summary_df = pd.merge(df_inter_group, df_intra_group, on=['veh_type','pollutantID','county'], suffixes=['_interzonal','_intrazonal'])
    summary_df['vmt'] = summary_df['vmt_interzonal']+summary_df['vmt_intrazonal']
    summary_df = pd.merge(summary_df, df_start_group, how='left', on=['veh_type','pollutantID','county'])
    # summary_df = finalize_emissions(summary_df, col_suffix="")
    pm10 = summary_df[summary_df['pollutantID'].isin([100,106,107])].groupby(['veh_type','county']).sum().reset_index()
    pm10['pollutantID'] = 'PM10'
    pm25 = summary_df[summary_df['pollutantID'].isin([110,116,117])].groupby(['veh_type','county']).sum().reset_index()
    pm25['pollutantID'] = 'PM25'
    summary_df = summary_df.append(pm10)
    summary_df = summary_df.append(pm25)
    summary_df.loc[~summary_df['pollutantID'].isin(['PM','PM10','PM25']),'pollutantID'] = summary_df[~summary_df['pollutantID'].isin(['PM','PM10','PM25'])]['pollutantID'].astype('int')
    summary_df['pollutant_name'] = summary_df['pollutantID'].astype('int', errors='ignore').astype('str').map(pollutant_map)
    summary_df['total_daily_tons'] = summary_df['start_tons']+summary_df['interzonal_tons']+summary_df['intrazonal_tons']
    summary_df = summary_df[['pollutantID','county','pollutant_name','veh_type','start_tons','intrazonal_tons','interzonal_tons','total_daily_tons','vmt']]

    # Filter for county
    if county:
        summary_df = summary_df[summary_df['county'] == county]

    return summary_df

In [44]:
for county in ['king','kitsap','pierce','snohomish']:    
    run_dir = r'\\modelstation2\c$\Workspace\sc_2018_rtp_final\soundcast'
    df_18_county = get_county_emissions(run_dir, county)

    run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2030_final\soundcast'
    df_30_county = get_county_emissions(run_dir, county)

    run_dir = r'\\modelstation1\c$\workspace\sc_2040_rtp_final\soundcast'
    df_40_county = get_county_emissions(run_dir, county)

    run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final\soundcast'
    df_50_county = get_county_emissions(run_dir, county)


    # Create interpolations
    df = df_18_county.merge(df_30_county, on=['pollutant_name','veh_type'], suffixes=['_2018','_2030'], how='outer')

    # Merge 2040
    year = '2040'
    df = df.merge(df_40_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)

    # # Merge 2050
    year = '2050'
    df = df.merge(df_50_county, on=['pollutant_name','veh_type'], how='outer')
    df.rename(columns={'total_daily_tons': 'total_daily_tons_'+year,
                        'vmt': 'vmt_'+year}, inplace=True)


    df_vmt = interpolate(df, 2018, 2030, 2019, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2020, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2021, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2022, 'vmt')
    df_vmt = interpolate(df, 2018, 2030, 2023, 'vmt')
    df_vmt = df_vmt[['pollutant_name','veh_type','vmt_2018','vmt_2019','vmt_2020',
                     'vmt_2020','vmt_2021','vmt_2022','vmt_2023','vmt_2030','vmt_2040','vmt_2050']]

    df_emissions = interpolate(df, 2018, 2030, 2019, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2020, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2021, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2022, 'total_daily_tons')
    df_emissions = interpolate(df, 2018, 2030, 2023, 'total_daily_tons')
    df_emissions = df_emissions[['pollutant_name','veh_type','total_daily_tons_2018', 'total_daily_tons_2019',
        'total_daily_tons_2020','total_daily_tons_2021','total_daily_tons_2022','total_daily_tons_2023',\
        'total_daily_tons_2030','total_daily_tons_2040','total_daily_tons_2050']]

    df_emissions = df_emissions[df_emissions['pollutant_name'].isin(['Atmospheric CO2','CO2 Equivalent','Methane','N20'])]
    df_emissions.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\\'+county+'_county_total_emissions.csv', index=False)

    _df_vmt = df_vmt.groupby('veh_type').first().reset_index()
    _df_vmt['geography'] = county.capitalize()+' County Total'
    _df_vmt.drop('pollutant_name', axis=1, inplace=True)
    _df_vmt.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\\'+county+'_county_total_vmt.csv', index=False)


In [46]:
# Join county totals together
county_summary_df = pd.DataFrame()
county_vmt_df = pd.DataFrame()
for county in ['king','kitsap','pierce','snohomish']:
    df = pd.read_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\\'+county+'_county_total_emissions.csv')
    df['county'] = county
    county_summary_df = county_summary_df.append(df)

    df = pd.read_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\\'+county+'_county_total_vmt.csv')
    df['county'] = county
    county_vmt_df = county_vmt_df.append(df)

county_summary_df.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\regional_emissions_by_county.csv', index=False)
county_vmt_df.to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\regional_vmt_by_county.csv', index=False)


In [80]:
king_total_df = county_summary_df[county_summary_df['county'] == 'king']
king_vmt_df = county_vmt_df[county_vmt_df['county'] == 'king']
# df_emissions


In [76]:
# df_city_tot_emissions

In [78]:
# 
df = df_city_tot_emissions.merge(king_total_df, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2021,2022,2023,2030,2040,2050]:
    df['total_daily_tons_unincorporated_'+str(year)] = df['total_daily_tons_'+str(year)+'_county_tot']-df['total_daily_tons_'+str(year)+'_cities_tot']

In [79]:
df[['pollutant_name','veh_type']+['total_daily_tons_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2021','2022','2023','2030','2040','2050']]].to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\king_county_unincorporated_emissions.csv', index=False)

In [81]:
df = king_vmt_df.drop('geography', axis=1).merge(df_city_tot_vmt, suffixes=['_county','_city_total'], on='veh_type')
# df = df_vmt.merge(df_emissions, suffixes=['_cities_tot','_county_tot'], on=['veh_type','pollutant_name'])
for year in [2018,2019,2020,2021,2022,2023,2030,2040,2050]:
    df['vmt_unincorporated_'+str(year)] = df['vmt_'+str(year)+'_county']-df['vmt_'+str(year)+'_city_total']

In [82]:
df['geography'] = 'Unincorporated King'
df[['veh_type','geography']+['vmt_unincorporated_'+str(year) for year in ['2018',
    '2019','2020','2021','2022','2023','2030','2040','2050']]].to_csv(r'R:\e2projects_two\data_requests\2024\king_county_emissions\request_10_08_24\king_county_unincorporated_vmt.csv', index=False)